In [1]:
%load_ext watermark
%watermark -a 'cs224' -u -d -v -p numpy,xarray,scipy,pandas,sklearn,matplotlib,seaborn,pymc3

/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/py36ds/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/py36ds/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


cs224 
last updated: 2020-03-05 

CPython 3.6.8
IPython 7.3.0

numpy 1.16.2
xarray 0.11.3
scipy 1.2.1
pandas 0.24.2
sklearn 0.20.3
matplotlib 3.0.3
seaborn 0.9.0
pymc3 3.8


In [2]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, scipy.special, scipy.misc, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, xarray as xr
import matplotlib as mpl

import pymc3 as pm

import theano as thno
import theano.tensor as T

import datetime, time, math
from dateutil import relativedelta

from collections import OrderedDict

SEED = 42
np.random.seed(SEED)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [3]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

# Model Comparison via Bayes Factor

* [Doing Bayesian Data Analysis: A Tutorial with R, JAGS, and Stan](https://www.amazon.de/Doing-Bayesian-Data-Analysis-Tutorial/dp/0124058884) p. 268

In [4]:
# tail-biased factory
omega1, kappa1 = 0.25, 12

In [5]:
# head-biased factory
omega2, kappa2 = 0.75, 12

In [6]:
# Flip coin 9 times and get 6 heads
samples = np.array([0,0,0,1,1,1,1,1,1])

In [7]:
def pD(z,N,a,b):
    #return scipy.special.beta(z+a, N-z+b)/scipy.special.beta(a,b)
    return np.exp(np.log(scipy.special.beta(z+a, N-z+b)) - np.log(scipy.special.beta(a,b)))

In [8]:
def ok2ab(omega, kappa):
    return omega*(kappa-2)+1,(1-omega)*(kappa-2)+1

In [9]:
a1,b1 = ok2ab(omega1,kappa1)
a2,b2 = ok2ab(omega2,kappa2)

In [10]:
p1 = pD(6,9,a1,b1)
p1

0.0004993438720703131

In [11]:
p2 = pD(6,9,a2,b2)
p2

0.0023385561429537273

In [12]:
def fn(a, b):
    return lambda p: stats.bernoulli(p).pmf(samples).prod() * stats.beta(a,b).pdf(p)

In [13]:
fn(1,1)(0.25)

0.000102996826171875

In [14]:
scipy.integrate.quad(fn(a1,b1),0.0,1.0)

(0.0004993438720702594, 2.322913347907592e-09)

In [15]:
p1

0.0004993438720703131

In [16]:
scipy.integrate.quad(fn(a2,b2),0.0,1.0)

(0.002338556142956198, 2.1998184481120156e-11)

In [17]:
p2

0.0023385561429537273

## Via PyMC3

* [Bayes Factors and Marginal Likelihood](https://docs.pymc.io/notebooks/Bayes_factor.html)

In [18]:
# Flip coin 9 times and get 6 heads
priors = ((a1, b1), (a2, b2))
models = []
traces = []
for alpha, beta in priors:
    with pm.Model() as model:
        a = pm.Beta('a', alpha, beta)
        yl = pm.Bernoulli('yl', a, observed=samples)
        trace = pm.sample_smc(4000, n_steps=100, random_seed=42)
        models.append(model)
        traces.append(trace)

Sample initial stage: ...
Stage:   0 Beta: 0.656 Steps: 100 Acce: 1.000
Stage:   1 Beta: 1.000 Steps: 100 Acce: 0.704
Sample initial stage: ...
Stage:   0 Beta: 1.000 Steps: 100 Acce: 1.000


In [19]:
p1

0.0004993438720703131

In [20]:
models[0].marginal_likelihood

0.0004979784247939404

In [21]:
p2

0.0023385561429537273

In [22]:
models[1].marginal_likelihood

0.002364542043803754

In [23]:
BF_smc = models[1].marginal_likelihood / models[0].marginal_likelihood
BF_smc

4.748282106362706

In [24]:
p2/p1

4.683257918552034